In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vigne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vigne\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [6]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('IntentFile').read()
intents = json.loads(data_file)

In [8]:
data_file

'{"intents": [\n        {"tag": "greeting",\n         "patterns": ["Hi there", "How are you", "Is anyone there?","Hey","Hola", "Hello", "Good day"],\n         "responses": ["Hello, thanks for asking", "Good to see you again", "Hi there, how can I help?"],\n         "context": [""]\n        },\n        {"tag": "goodbye",\n         "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time"],\n         "responses": ["See you!", "Have a nice day", "Bye! Come back again soon."],\n         "context": [""]\n        },\n        {"tag": "thanks",\n         "patterns": ["Thanks", "Thank you", "That\'s helpful", "Awesome, thanks", "Thanks for helping me"],\n         "responses": ["Happy to help!", "Any time!", "My pleasure"],\n         "context": [""]\n        },\n        {"tag": "noanswer",\n         "patterns": [],\n         "responses": ["Sorry, can\'t understand you", "Please give me more info", "Not sure I understand"],\n         "context": [""]\n        },

In [9]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [10]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [16]:
classes = sorted(list(set(classes)))

In [14]:
print (len(documents), "documents")

47 documents


In [13]:
print (len(classes), "classes", classes)


9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [12]:
print (len(words), "unique lemmatized words", words)

88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [17]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [18]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [19]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


In [20]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [21]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [23]:
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
47/47 [==============================] - 0s 395us/step - loss: 0.0123 - accuracy: 1.0000
Epoch 2/200
47/47 [==============================] - 0s 328us/step - loss: 0.0149 - accuracy: 1.0000
Epoch 3/200
47/47 [==============================] - 0s 367us/step - loss: 0.0054 - accuracy: 1.0000
Epoch 4/200
47/47 [==============================] - 0s 351us/step - loss: 0.0083 - accuracy: 1.0000
Epoch 5/200
47/47 [==============================] - 0s 399us/step - loss: 0.0065 - accuracy: 1.0000
Epoch 6/200
47/47 [==============================] - 0s 351us/step - loss: 0.0133 - accuracy: 1.0000
Epoch 7/200
47/47 [==============================] - 0s 326us/step - loss: 0.0240 - accuracy: 1.0000
Epoch 8/200
47/47 [==============================] - 0s 376us/step - loss: 0.0044 - accuracy: 1.0000
Epoch 9/200
47/47 [==============================] - 0s 451us/step - loss: 0.0038 - accuracy: 1.0000
Epoch 10/200
47/47 [==============================] - 0s 396us/step - loss: 0.0013 - accura